# Mnist classification demo

In [1]:
%%capture
import grpc 
import numpy as np
import hydro_serving_grpc as hs
import keras
from keras.datasets import mnist
from keras import backend as K
import os

## Read data sample

In [ ]:
Xtest = np.load('activity_recognition/data/SHL5hip_300_X_test.npy')[:,0]
val = np.array(Xtest)[1].reshape(1,300,3,3).astype(np.float32)
val = val.flatten()

## Setup channel

In [6]:
channel = grpc.insecure_channel("localhost") 
stub = hs.PredictionServiceStub(channel) 
model_spec = hs.ModelSpec(name="activity_recognition", signature_name="infer")

## Create tensors

In [7]:
tensor_shape = hs.TensorShapeProto(dim=[hs.TensorShapeProto.Dim(size=1), hs.TensorShapeProto.Dim(size=300),hs.TensorShapeProto.Dim(size=3),hs.TensorShapeProto.Dim(size=3)])
tensor = hs.TensorProto(dtype=hs.DT_DOUBLE, tensor_shape=tensor_shape, double_val=val)

## Send request to model servable

In [8]:
request = hs.PredictRequest(model_spec=model_spec, inputs={"x": tensor})
result = stub.Predict(request)
result = np.array(result.outputs['y']).argmax()
print('Output :')
print(f'activity # :{result}')

Output values:
outputs {
  key: "y"
  value {
    dtype: DT_DOUBLE
    tensor_shape {
      dim {
        size: 5
      }
    }
    double_val: 0.003309800988063216
    double_val: 0.9832870960235596
    double_val: 0.010390128009021282
    double_val: 0.00010289066995028406
    double_val: 0.0029099872335791588
  }
}

